# Check The Status of Model
Memeriksa status model yang telah dideploy pada server.

In [1]:
import requests
from pprint import PrettyPrinter
 
pp = PrettyPrinter()
pp.pprint(requests.get("https://hoax-model-production.up.railway.app/v1/models/hoax-model").json())

{'model_version_status': [{'state': 'AVAILABLE',
                           'status': {'error_code': 'OK', 'error_message': ''},
                           'version': '1'}]}


# Make Prediction Request

## Import Library
Mengimpor library yang dibutuhkan untuk membuat permintaan prediksi.

In [2]:
import requests
import tensorflow as tf
import base64

## Make functions for transform text
Membuat fungsi untuk mengubah teks menjadi format yang sesuai untuk input model.

In [3]:
def _bytes_feature(value):
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def serialize_text(text):
    example = tf.train.Example(features=tf.train.Features(feature={
        'narasi': _bytes_feature(text)
        }))
    serialized_example = example.SerializeToString()
    return serialized_example

## Make function for predict
Mendefinisikan URL server model dan membuat fungsi untuk melakukan prediksi dengan menggunakan model yang telah dideploy.

In [4]:
model_server_url = 'https://hoax-model-production.up.railway.app/v1/models/hoax-model:predict'

def predict(text):
    example = serialize_text(text)
    json_data = {
      "signature_name":"serving_default",
      "instances":[
        {
          "examples":{"b64": base64.b64encode(example).decode('utf-8')}
        }
      ]
    }
    resp = requests.post(model_server_url, json=json_data)
    if resp.status_code != 200:
        print(f"Error: {resp.status_code} - {resp.text}")
    return resp.json()

## Test Predictions
Menyiapkan beberapa teks untuk diuji dan memanggil fungsi prediksi untuk mengevaluasi apakah teks tersebut terdeteksi sebagai Hoax atau tidak.

In [5]:
texts = [b'Saya Harry Mardika akan maju menjadi Presiden Indonesia Emas 2045', # HOAX
        b'Presiden Jokowi mengadakan kunjungan kerja '] # NOT HOAX

for text in texts:
    pred = predict(text)
    decoded_text = text.decode('utf-8')
    if pred['predictions'][0][0] > 0.5:
        print('Score Prediction :',pred['predictions'][0][0])
        print(f"Kalimat : '{decoded_text}'")
        print('Terdeteksi sebagai Hoax\n')
    else:
        print('Score Prediction :',pred['predictions'][0][0])
        print(f"Kalimat : '{decoded_text}'")
        print('Tidak terdeteksi sebagai Hoax\n')


Score Prediction : 0.991513848
Kalimat : 'Saya Harry Mardika akan maju menjadi Presiden Indonesia Emas 2045'
Terdeteksi sebagai Hoax

Score Prediction : 0.176164046
Kalimat : 'Presiden Jokowi mengadakan kunjungan kerja '
Tidak terdeteksi sebagai Hoax

